# **Data Acquisition**

## Notebook Setup

First, import all necessary packages

In [1]:
!pip install pymannkendall xarray cfgrib contextily cartopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 16.0 MB/s eta 0:00:00


In [2]:
import ee
import geemap
import geemap.colormaps as cm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import math
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.sparse import diags, eye
from scipy.sparse.linalg import spsolve
from scipy.signal import savgol_filter
import pymannkendall as mk
import json
import os
from tqdm.notebook import tqdm
import geopandas as gpd
from glob import glob
from shapely.geometry import shape
from matplotlib.colors import TwoSlopeNorm
import contextily as cx  # for adding basemaps
import cartopy.crs as ccrs
from cartopy.io.img_tiles import OSM
import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize
from matplotlib import cm
from shapely.geometry import shape, mapping
import xarray as xr

Set user defined variables

In [ ]:
setup_path = '/content/drive/MyDrive/final_project_code/src/analysis_setup.txt'

config = {}

with open(setup_path, 'r') as f:

    for line in f:
        line = line.strip()
        if not line or line.startswith('#'):
            continue
        key, value = line.split('=', 1)
        key = key.strip()
        value = value.strip().strip("'").strip('"')
        config[key] = value

# Convert known integers
config['start_year'] = int(config['start_year'])
config['end_year'] = int(config['end_year'])

# Create the variables from the config dictionary
ee_username = config['ee_username']
study_area = config['study_area']
start_date = config['start_date']
end_date = config['end_date']
start_year = config['start_year']
end_year = config['end_year']
raw_pixel_evi_data = config['raw_pixel_evi_data']
processed_pixel_evi_data = config['processed_pixel_evi_data']
greening_strip = config['greening_strip']
kg_zones = config['kg_zones']
correlation_data = config['correlation_data']
geo_tiff_data = config['geo_tiff_data']

# Print them for reference and verification
print(ee_username, type(ee_username))
print(study_area, type(study_area))
print(start_date, type(start_date))
print(end_date, type(end_date))
print(start_year, type(start_year))
print(end_year, type(end_year))
print(raw_pixel_evi_data, type(raw_pixel_evi_data))
print(processed_pixel_evi_data, type(processed_pixel_evi_data))
print(greening_strip, type(greening_strip))
print(kg_zones, type(kg_zones))
print(correlation_data, type(correlation_data))
print(geo_tiff_data, type(geo_tiff_data))

ee-pantonopoulos517 <class 'str'>
projects/ee-pantonopoulos517/assets/Peru_Chile_ROI-2 <class 'str'>
2000-01-01 <class 'str'>
2024-12-31 <class 'str'>
2000 <class 'int'>
2024 <class 'int'>
/content/drive/MyDrive/final_raw_data <class 'str'>
/content/drive/MyDrive/final_processed_data <class 'str'>
projects/ee-pantonopoulos517/assets/final_greening_strip <class 'str'>
projects/ee-pantonopoulos517/assets/koppen_geiger_0p00833333 <class 'str'>
/content/drive/MyDrive/correlation_data <class 'str'>
/content/drive/MyDrive/geo_tiff_data <class 'str'>


Initialise Google Earth Engine

In [ ]:
try:
    ee.Initialize(project=ee_username)
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project=ee_username)

## Data Acquisition Pipeline

First we need to tile the Region Of Interest (ROI) so we can work in batches. Generate the tiles in the MODIS projection, so it matches the data exactly

In [ ]:
# Load ROI
roi = ee.FeatureCollection(study_area)

# MODIS EVI collection with SummaryQA
modis_collection = ee.ImageCollection("MODIS/061/MOD13Q1") \
    .select(["EVI", "SummaryQA"]) \
    .filterDate(start_date, end_date) \
    .filterBounds(roi)

In [ ]:
def generate_modis_tiles(roi, tile_size_m=50000):  # Approx 50 sqkm tiles
    """
    Generate a grid of MODIS-native tiles over a region of interest (ROI).

    This function divides the given ROI into square tiles of a specified size
    (default is 50,000 meters, approx. 50 km), using the MODIS Sinusoidal projection 
    (SR-ORG:6974). It returns a list of tiles that intersect with the ROI.

    Parameters:
        roi (ee.Feature or ee.Geometry): 
            The region of interest over which to generate the tiles.
        tile_size_m (int, optional): 
            The size of each tile in meters (default is 50,000).

    Returns:
        List[dict]: A list of dictionaries, each representing a tile with:
            - 'i': column index
            - 'j': row index
            - 'geometry': an ee.Geometry.Rectangle in MODIS projection
    """

    modis_proj = ee.Projection('SR-ORG:6974')  # MODIS sinusoidal projection

    # Transform ROI to MODIS projection
    roi_proj_geom = roi.geometry().transform(modis_proj, 100)
    roi_coords = roi_proj_geom.coordinates().get(0)

    # Get min and max coordinates of the transformed ROI
    coords_list = ee.List(roi_coords)
    xs = coords_list.map(lambda pt: ee.List(pt).get(0))
    ys = coords_list.map(lambda pt: ee.List(pt).get(1))
    min_x = ee.Number(xs.reduce(ee.Reducer.min()))
    max_x = ee.Number(xs.reduce(ee.Reducer.max()))
    min_y = ee.Number(ys.reduce(ee.Reducer.min()))
    max_y = ee.Number(ys.reduce(ee.Reducer.max()))

    # Calculate tile counts
    x_tiles = ee.Number(max_x.subtract(min_x)).divide(tile_size_m).ceil().getInfo()
    y_tiles = ee.Number(max_y.subtract(min_y)).divide(tile_size_m).ceil().getInfo()

    tiles = []

    for i in range(int(x_tiles)):
        for j in range(int(y_tiles)):
            x0 = min_x.getInfo() + i * tile_size_m
            y0 = min_y.getInfo() + j * tile_size_m
            x1 = x0 + tile_size_m
            y1 = y0 + tile_size_m

            tile_geom = ee.Geometry.Rectangle([x0, y0, x1, y1], proj=modis_proj, geodesic=False) # Make rectangle in MODIS projection

            if tile_geom.intersects(roi_proj_geom, ee.ErrorMargin(100)).getInfo():
                tiles.append({
                    "i": i,
                    "j": j,
                    "geometry": tile_geom
                })

    print(f"Generated {len(tiles)} MODIS-native tiles (~{tile_size_m/1000:.0f} km).")
    return tiles

In [ ]:
# Generate tiles (list of dictionaries)
tiles = generate_modis_tiles(roi, tile_size_m=50000)

# Convert to Earth Engine FeatureCollection
tile_features_geom = [ee.Feature(tile["geometry"]) for tile in tiles]
fine_tiles_fc = ee.FeatureCollection(tile_features_geom)

Let's visualise the tiles. We should see that they follow the MODIS sinusoidal projection

In [ ]:
Map = geemap.Map(center=[-9.19, -75.02], zoom=7)
Map.addLayer(roi, {"color": "blue"}, "ROI")
Map.addLayer(fine_tiles_fc.style(color='red', fillColor='00000000'), {}, "Fine MODIS Tiles")
Map.addLayerControl()
Map

Now for all tiles, we need to export all the contained pixel's EVI time series and SummaryQA values

In [ ]:
def export_evi_qa_time_series(tile, modis_collection, tile_id):
    """
    Export per-pixel time series of MODIS EVI and QA data for a given tile.

    This function processes an Earth Engine tile geometry by extracting EVI and 
    SummaryQA bands from a MODIS image collection, organising them as time stamped 
    bands, and exporting the sampled pixel values as a CSV file to Google Drive.

    Parameters:
        tile (dict): 
            A dictionary representing a tile, with at least a 'geometry' key 
            containing an `ee.Geometry` object in MODIS sinusoidal projection.
        modis_collection (ee.ImageCollection): 
            The MODIS image collection filtered to include 'EVI' and 'SummaryQA' bands.
        tile_id (int): 
            A unique integer ID used to name the export files and track progress.
    """
    sample_geom = tile['geometry']
    clip_geom = sample_geom

    filtered = modis_collection.filterBounds(sample_geom)

    # Rename EVI bands with timestamp
    def rename_evi(image):
        date_str = ee.Date(image.get('system:time_start')).format('YYYYMMdd')
        name = ee.String('EVI_').cat(date_str)
        return image.select(['EVI']).rename([name])

    evi_stack = filtered.map(rename_evi)
    evi_bands = ee.ImageCollection(evi_stack).toBands()

    # Rename SummaryQA bands with timestamp
    def rename_qa(image):
        date_str = ee.Date(image.get('system:time_start')).format('YYYYMMdd')
        name = ee.String('QA_').cat(date_str)
        return image.select(['SummaryQA']).rename([name])

    qa_stack = filtered.map(rename_qa)
    qa_bands = ee.ImageCollection(qa_stack).toBands()

    # Combine bands
    combined = evi_bands.addBands(qa_bands)

    # Align to MODIS sinusoidal grid 
    combined = combined.reproject(crs='SR-ORG:6974', scale=250)

    # Take all pixels in each tile using .sample() with a very large numPixels (upper limit)
    samples = combined.sample(
        region=sample_geom,
        scale=250,
        geometries=True,
        seed=42,
        numPixels=1e9  
    ).map(lambda f: f.setGeometry(f.geometry().intersection(clip_geom, 1)))

    # Export to Drive as .csv
    task = ee.batch.Export.table.toDrive(
        collection=samples,
        description=f"EVI_QA_PerPixel_Tile_{tile_id}",
        folder="final_raw_data",
        fileNamePrefix=f"tile_{tile_id}",
        fileFormat="CSV"
    )
    task.start()

    if tile_id % 50 == 0:
        print(f"Started export for tile {tile_id}")

In [ ]:
for i, tile in enumerate(tiles):
    export_evi_qa_time_series(tile, modis_collection, i)

Now we have exported all tiles covering the ROI, where each tile csv file includes all pixels contained within that tile. The file's rows represent each pixel. Each file includes the whole EVI time series for each pixel together with the SummaryQA band which is used to identify snow/ice and clouds